In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from parse import *
import sqlalchemy as sql
import time 

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
company_df = pd.read_sql_query(query, sql_engine)

total_data = []
for x in range(0, len(company_df)):
    for fin_typ in ['3', '4']:
        code = company_df.iloc[x]['종목코드']
        name = company_df.iloc[x]['종목명']

        url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930'
        html_text = requests.get(url).text
        encparam = re.findall ("encparam: '(.*?)'", html_text)[0]

        url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?' \
                        'cmp_cd={cmp_cd}&fin_typ={fin_typ}&freq_typ={freq_typ}&encparam={encparam}&id={id}'  

        query={
            'cmp_cd': code,
            'fin_typ': fin_typ,
            'freq_typ': 'A',
            'encparam': encparam,
            'id': 'ZTRzQlVCd0'
                }

        url = url_tmpl.format(cmp_cd=query['cmp_cd'], fin_typ=query['fin_typ'], freq_typ=query['freq_typ'], encparam=query['encparam'], id=query['id'])
        headers={'Referer': url}
        req = requests.get(url, headers=headers).text
        soup = BeautifulSoup(req, 'html.parser')

        try:
            table = soup.findAll("table", {"class": "gHead01 all-width"})[0]
            dfs = pd.read_html(str(table))[0]
            column_list = list(dfs.columns)
            for i in column_list[1:]:
                try:
                    Gubun_result = parse("{}컨센서스보기", i[0])
                    Date_result = parse("{}  ({})", i[1])
                    Gubun = Gubun_result[0][:2]
                    Date = Date_result[0][:7]
                    Date = pd.to_datetime(Date)
                    Date_gubun = Date_result[0][7:]
                    if Date_gubun == '(E)':
                        Date_gubun = '컨센서스'
                    else:
                        Date_gubun = '실적'
                    Acc_type = Date_result[1]
                    if Acc_type == 'IFRS별도':
                        Acc_type = 'KIFRS별도'
                    if Gubun == '연간':
                        result_list = [code, name, Gubun, Acc_type, Date.year, np.nan, Date_gubun]
                    elif Gubun == '분기':
                        result_list = [code, name, Gubun, Acc_type, Date.year, Date.quarter, Date_gubun]
                    for j in dfs[i]:
                        result_list.append(j)
                    total_data.append(result_list)
                except:
                    pass
        except:
            pass
        

column_list = list(dfs.columns)
columns_name = list(dfs[column_list[0]])
columns_name = ['종목코드', '종목명', '구분', '회계', '년도', '분기', '컨센서스여부'] + columns_name

df = pd.DataFrame(total_data, columns= columns_name)
df['차입금비율'] = df['이자발생부채']*100/df['자본총계(지배)']
df = df.replace([np.inf, -np.inf], np.nan)

df.insert(loc=0, column='No', value=np.nan)

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(2),
    '회계': sql.types.VARCHAR(7),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '컨센서스여부': sql.types.VARCHAR(4)
}

for i in list(df.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 

df_year = df.loc[df['구분'] == '연간']
df_quarter = df.loc[df['구분'] == '분기']

No_list = []
for i in range(0, len(df_year)):
    No_list.append(i+1)
df_year['No'] = No_list

No_list = []
for i in range(0, len(df_quarter)):
    No_list.append(i+1)
df_quarter['No'] = No_list

df_year.to_sql(name="NAVER_FINANCE_SUMMARY_연간데이터", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `NAVER_FINANCE_SUMMARY_연간데이터` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `NAVER_FINANCE_SUMMARY_연간데이터` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `NAVER_FINANCE_SUMMARY_연간데이터` ADD INDEX(`종목코드`);')
    
    
df_quarter.to_sql(name="NAVER_FINANCE_SUMMARY_분기데이터", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `NAVER_FINANCE_SUMMARY_분기데이터` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `NAVER_FINANCE_SUMMARY_분기데이터` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `NAVER_FINANCE_SUMMARY_분기데이터` ADD INDEX(`종목코드`);')
    
now = time.localtime()
current_time = "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

print("[" + current_time + "] " + "NAVER_FINANCE_SUMMARY 테이블 업데이트 완료하였습니다.")

        주요재무정보        연간연간컨센서스보기                                      \
        주요재무정보 2017/12  (IFRS별도) 2018/12  (IFRS별도) 2019/12  (IFRS별도)   
0          매출액           2589.00           3066.00           3072.00   
1         영업이익            110.00            112.00            101.00   
2   영업이익(발표기준)            110.00            112.00            101.00   
3     세전계속사업이익            652.00            145.00            167.00   
4        당기순이익            470.00            101.00             95.00   
5    당기순이익(지배)            470.00            101.00             95.00   
6   당기순이익(비지배)               NaN               NaN               NaN   
7         자산총계           3672.00           3703.00           3765.00   
8         부채총계            703.00            732.00            749.00   
9         자본총계           2969.00           2971.00           3016.00   
10    자본총계(지배)           2969.00           2971.00           3016.00   
11   자본총계(비지배)               NaN               NaN              

        주요재무정보        연간연간컨센서스보기                                      \
        주요재무정보 2017/12  (IFRS별도) 2018/12  (IFRS별도) 2019/12  (IFRS별도)   
0          매출액           2881.00           2791.00           2536.00   
1         영업이익            394.00            390.00            390.00   
2   영업이익(발표기준)            394.00            390.00            390.00   
3     세전계속사업이익            359.00            362.00            325.00   
4        당기순이익            257.00            292.00            222.00   
5    당기순이익(지배)            257.00            292.00            222.00   
6   당기순이익(비지배)               NaN               NaN               NaN   
7         자산총계          11991.00          12265.00          12963.00   
8         부채총계           4881.00           4974.00           5555.00   
9         자본총계           7110.00           7292.00           7407.00   
10    자본총계(지배)           7110.00           7292.00           7407.00   
11   자본총계(비지배)               NaN               NaN              

        주요재무정보        연간연간컨센서스보기                                      \
        주요재무정보 2017/12  (IFRS별도) 2018/12  (IFRS별도) 2019/12  (IFRS별도)   
0          매출액            916.00            831.00            581.00   
1         영업이익            428.00            457.00            197.00   
2   영업이익(발표기준)            428.00            457.00            197.00   
3     세전계속사업이익            417.00            485.00            223.00   
4        당기순이익            412.00            484.00            206.00   
5    당기순이익(지배)            412.00            484.00            206.00   
6   당기순이익(비지배)               NaN               NaN               NaN   
7         자산총계          15781.00          15907.00          16080.00   
8         부채총계           2067.00           2035.00           2214.00   
9         자본총계          13714.00          13872.00          13866.00   
10    자본총계(지배)          13714.00          13872.00          13866.00   
11   자본총계(비지배)               NaN               NaN              

        주요재무정보        연간연간컨센서스보기                                      \
        주요재무정보 2017/12  (IFRS별도) 2018/12  (IFRS별도) 2019/12  (IFRS별도)   
0          매출액          16649.00          16576.00          18301.00   
1         영업이익            835.00            900.00            803.00   
2   영업이익(발표기준)            835.00            900.00            803.00   
3     세전계속사업이익            369.00            428.00             33.00   
4        당기순이익            186.00            249.00           -407.00   
5    당기순이익(지배)            186.00            249.00           -407.00   
6   당기순이익(비지배)               NaN               NaN               NaN   
7         자산총계          34034.00          33167.00          31320.00   
8         부채총계          21312.00          21003.00          20502.00   
9         자본총계          12722.00          12164.00          10818.00   
10    자본총계(지배)          12722.00          12164.00          10818.00   
11   자본총계(비지배)               NaN               NaN              

KeyboardInterrupt: 